# 네이버 뉴스 API를 사용해서 요양원의 사회 이슈를 파악합니다.
[naver api 문서](https://developers.naver.com/docs/common/openapiguide/)  
네이버 뉴스 api url : https://openapi.naver.com/v1/search/news  
GET형식으로 JSON, XML형식의 파일을 반환합니다. 반환내요은 네이버 검색의 뉴스 검색 결과를 반환합니다.  
## naverapi 응답 구조
* lastBuildDate : api 요청한 시각  
* total : 관련 뉴스 수  
* start : 시작  
* display : 가져온 갯수  
* items : 뉴스  
<items 구조>    
title : 뉴스 제목  
originallink : 원래의 뉴스 주소
link : 네이버 뉴스 링크  
description : 뉴스 내용  
pubDate : 출판날짜 

## naverapi 개발자 key와 비밀번호 받는 방법
[naver 문서](https://developers.naver.com/docs/common/openapiguide/appregister.md)


# 1. naverapi로 뉴스 200개 가져오기

In [ ]:
import urllib.request
import json
import pandas as pd
import re
import time
from konlpy.tag import Kkma
import numpy as np

In [ ]:
# 네이버 ReatAPI
def naverNews(query, num) :
    client_key = "LlQltlguC0ebo54G0laV"
    client_secret = "2iRS_csaj1"
    encText = urllib.parse.quote_plus(query)
    count = [[i,100] for i in range(1, num, 100)]
    if num % 100:
        count[-1][1] = num % 100 
    for i in count:
        naver_url = 'https://openapi.naver.com/v1/search/news?query=' + encText + '&display=' + str(i[1]) +'&start=' + str(i[0])
        # request 객체에 add하기
        request = urllib.request.Request(naver_url)
        request.add_header("X-Naver-Client-Id",client_key)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        # 응답받은 코드가 정상적인지 확인하기
        rescode = response.getcode()
        if(rescode == 200):
            response_body = response.read()
            if i[0] == 1:
                dataList = json.loads(response_body)
            else:
                dataList["items"] = dataList["items"] + json.loads(response_body)["items"]
        else:
            print('오류 코드 : ' + rescode)
    return dataList

In [ ]:
# API로 응답받은 데이터중에 사용할 데이터만 전처리
def itemsToDf(dataList):
    itemsdf = pd.DataFrame(dataList["items"])
    for i in range(itemsdf.shape[0]):
        # title열 데이터 정리하기
        title = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"title"]))
        title = re.sub("&.{1,5};", "", title)
        title = re.sub(" ( )+", "", title)
        itemsdf.loc[i,"title"] = title
        # description열 데이터 정리하기
        description = re.sub("<b>|</b>", " ", str(itemsdf.loc[i,"description"]))
        description = re.sub("&.{1,5};", "", description)
        description = re.sub(" ( )+", "", description)
        itemsdf.loc[i,"description"] = description
    return itemsdf

In [ ]:
data = naverNews("요양원", 200)
data = itemsToDf(data)

In [ ]:
# data/nver_crawl/검색년도-검색월-검색일.csv로 저장하기
searchDate = time.strftime('%Y-%m-%d', time.localtime(time.time()))
data.to_csv("data/naver_crawl/" + searchDate + ".csv", index=False)

# 2. 파일로 저장된 정보로 분석하기
https://blog.winterjung.dev/2017/04/21/list-of-lists-to-flatten  


In [ ]:
from konlpy.tag import Kkma
sample = '이것은 형태소 분석기 입니다 아버지가방에들어가신다'
kkma = Kkma()
print(kkma.nouns(sample))
# print(kkma.morphs(sample))
# print(kkma.pos(sample))

In [ ]:
# 뉴스 제목만 분석하기
# 오늘날짜 파일 불러오기
file = "data/naver_crawl/" + time.strftime('%Y-%m-%d', time.localtime(time.time())) + ".csv"
data = pd.read_csv(file)
# 제목 형태소 분석하기
nounsList = []
kkma = Kkma()
for i in range(len(data)):
    nonusTemp = kkma.nouns(data.loc[i, "title"])
    nounsList.append(nonusTemp)

In [ ]:
# 이중리스트 풀기
nounsList = [y for x in nounsList for y in x]